## Bogio Scrapper for Warcraft Logs - Verify your rotation against SimC - Frost Mage 10.1.5
## NEED TO CHANGE FOR C PATH

### Make sure to install any required libraries below


In [1]:
#add any other libraries you need
%pip install lxml
%pip install requests
%pip install bs4
%pip install pandas
%pip install openpyxl
%pip install selenium


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Normally we'd explore the data and parse it with soup but the rotation data is dynamic so we can't automatically extract it the easy way. The information we need is in the "timeline-lines" and as you can check, it is empty.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_spell_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        return soup  # Return the 'soup' object

url = 'https://www.warcraftlogs.com/reports/HNTmQ4wBYypRPVh2/#fight=last&type=casts&source=110&start=743381&end=1056565&view=timeline'
soup = scrape_spell_data(url)  # Assign the returned 'soup' object to a variable


# Now you can use 'soup' to explore and extract data
print(soup.prettify())




### Instructions how to prepare your data manually
1. Upload your logs
2. Edit the timeline and casts as you want
3. Press F12
4. Go into elements
5. Ctrl+F and type "timeline-lines"
6. If you mouseover and the window highlights - it means you are in a good spot
7. Right click, copy -> copy element and save the content into "raw_data.txt" file

![Image Description](Prepare_data.jpg)





### Or you can install chromedriver and automate it with selenium
1. Upload your logs
2. Edit the timeline and other data
3. copy the url and paste it below







In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

def scrape_dynamic_website(url):
    # Create a Chrome WebDriver
    driver = webdriver.Chrome()

    # Navigate to the URL
    driver.get(url)

    # Wait for the dynamic content to load (you may need to adjust the time)
    driver.implicitly_wait(10)

    # Find and extract the dynamic elements
    table_element = driver.find_element(By.CLASS_NAME, "timeline-lines")
    table_content = table_element.get_attribute('outerHTML')

    # Manually specify the path to the "bin" folder
    bin_folder = os.path.join(os.getcwd(), "bin")

    # Save the dynamic content to a .txt file in the "bin" folder
    file_path = os.path.join(bin_folder, 'raw_data.txt')
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(table_content)

    # Close the WebDriver
    driver.quit()

    print("Dynamic content has been copied to 'raw_data.txt' in the 'bin' folder.")

# Replace with the URL of the website you want to scrape
url = 'https://www.warcraftlogs.com/reports/HNTmQ4wBYypRPVh2/#fight=last&type=casts&source=110&start=743381&end=1056565&view=timeline'
scrape_dynamic_website(url)

Dynamic content has been copied to 'raw_data.txt' in the 'bin' folder.


### Lets extract the "timestamps" and skills once you get your data prepared


In [2]:
import pandas as pd
import re

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def extract(file_content, search_string, char_length):
    timestamps = []
    start_index = 0
    
    while True:
        # Find the occurrence of the search string in the content
        start_index = file_content.find(search_string, start_index)
        
        if start_index == -1:
            # If no more occurrences are found, break the loop
            break
        
        # Extract the timestamp (next 7 characters after the search string)
        timestamp = file_content[start_index + len(search_string): start_index + len(search_string) + char_length]
        
        # Remove commas from the timestamp and append to the list
        timestamps.append(timestamp.replace(',', ''))
        
        # Move the start_index to continue searching from the next position
        start_index += len(search_string)
    
    return timestamps

file_path = r'C:\Python_Projects\Rotation_Analyzer_WoW\bin\raw_data.txt'
file_content = read_file(file_path)
search_timestamp = '<div onmouseover="showTimelineTooltipText(this, 1, printEvent({&quot;timestamp&quot;:'
search_src = 'abilities/'
timestamps = extract(file_content, search_timestamp, 7)
src = extract(file_content, search_src, 25)

#print(timestamps)
#timestamps
#print(src)
data = {'timestamps': timestamps, 'Name': src}
df = pd.DataFrame(data)
df.head(10)
#df.dtypes



,timestamps,Name
0,747054,spell_frost_frostbolt02.j
1,762733,spell_frost_frostbolt02.j
2,763750,spell_frost_frostbolt02.j
3,769066,spell_frost_frostbolt02.j
4,774612,spell_frost_frostbolt02.j
5,778530,spell_frost_frostbolt02.j
6,779446,spell_frost_frostbolt02.j
7,783269,spell_frost_frostbolt02.j
8,784957,spell_frost_frostbolt02.j
9,788932,spell_frost_frostbolt02.j


### Lets sort the data

In [3]:
#sort the data
df['timestamps'] = pd.to_numeric(df['timestamps'])
#df.dtypes
# Sort the DataFrame by the "timestamps" column in ascending order
df_sorted = df.sort_values(by='timestamps', ascending=True)

df_sorted.head(10)

,timestamps,Name
0,747054,spell_frost_frostbolt02.j
135,748857,ability_warlock_burningem
252,748857,"racial_troll_berserk.jpg"""
256,748857,trade_alchemy_dpotion_b20
249,748857,spell_frost_coldhearted.j
254,749062,ability_mage_timewarp.jpg
224,749669,spell_mage_cometstorm2.jp
70,750504,spell_frost_frostblast.jp
234,751337,ability_mage_rayoffrost.j
71,753811,spell_frost_frostblast.jp


### Lets check how many times each spell was used

In [4]:
df_spell_density=df.groupby("Name")
df_spell_density.size()
df_spell_counts = df_spell_density.size().reset_index(name='Count')
df_spell_counts_sorted = df_spell_counts.sort_values(by='Count', ascending=False)
df_spell_counts_sorted.head(10)

,Name,Count
9,spell_frost_frostbolt02.j,70
8,spell_frost_frostblast.jp,65
5,ability_warlock_burningem,38
1,ability_mage_glacialspike,35
0,ability_ardenweald_mage.j,20
11,spell_mage_cometstorm2.jp,10
3,ability_mage_rayoffrost.j,6
10,spell_frost_frozenorb.jpg,5
7,spell_frost_coldhearted.j,3
2,ability_mage_greaterinvis,2


### And lets save the data to a new csv

In [5]:
import os
import pandas as pd

# Get the current working directory
current_directory = os.getcwd()

# Specify the folder where you want to save the Excel file (e.g., "bin")
save_folder = "bin"

# Combine the current directory and the save folder to get the full path
save_path = os.path.join(current_directory, save_folder)

# Create the "bin" folder if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Specify the new Excel file name
excel_file_new = os.path.join(save_path, "Rotation_WarcraftLogs.xlsx")

# Save the df_sorted DataFrame to the new Excel file with the specified sheet name and starting from cell A1
with pd.ExcelWriter(excel_file_new, engine='openpyxl') as writer:
    df_sorted.to_excel(writer, sheet_name="Warcraft_Logs", index=False)

    # Specify the starting cell for df_spell_counts_sorted
    start_row = 0
    start_col = 5  # This will be the column after the last column of df_sorted

    # Save the df_spell_counts_sorted DataFrame to the same Excel file and sheet
    df_spell_counts_sorted.to_excel(writer, sheet_name="Warcraft_Logs", startrow=start_row, startcol=start_col, index=False)


In [4]:
import sys
import os
script_path = os.getcwd()
excel_path = os.path.dirname(script_path)
script_path_filename = os.path.abspath(os.getcwd())
script_path


'c:\\Python_Projects\\Rotation_Analyzer_WoW\\bin'